# 2 Layer Neural Network

## Imports

In [1]:
import os
import pandas as pd

## Import data and create .csv

In [2]:
# Directory to MRI Images
base_dir = '/Users/benrandoing/Downloads/archive/AugmentedAlzheimerDataset'
classes = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

if not os.path.exists('dataset.csv'):
    data = []

    for label, class_name in enumerate(classes):
        class_dir = os.path.join(base_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            data.append([image_path, label])

    df = pd.DataFrame(data, columns=['path', 'label'])
    df.to_csv('dataset.csv', index=False)

## Data Loading and Pre-Processing

In [3]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.dataframe = pd.read_csv(csv_file)
        self.dataframe = self.dataframe.sample(frac=1).reset_index(drop=True)  # Shuffle the dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name)
        label = int(self.dataframe.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224 X 224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Load the entire dataset
dataset = CustomDataset(csv_file='dataset.csv', transform=transform)

# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [4]:
print(f'Training Samples: {len(train_dataset)}')
print(f'Validation Samples: {len(val_dataset)}')
print(f'Test Samples: {len(test_dataset)}')

Training Samples: 23788
Validation Samples: 5097
Test Samples: 5099


In [5]:
print(train_dataset)

In [6]:
import matplotlib.pyplot as plt
#plt.plot(train_dataset[0][0])
#plt.show()

## Pytorch Logistic Regression Model

In [7]:
class FiveLayerNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim):
        super(FiveLayerNN, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim1)
        self.fc2 = torch.nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = torch.nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = torch.nn.Linear(hidden_dim3, hidden_dim4)
        self.fc5 = torch.nn.Linear(hidden_dim4, output_dim)

    def forward(self, x):
        #x = x.view(-1, 224 * 224 * 3)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

input_dim = 224 * 224 * 3  # Assuming the images are RGB
hidden_dim1 = 512
hidden_dim2 = 256
hidden_dim3 = 128
hidden_dim4 = 64
output_dim = 4  # Four classes

#model = FiveLayerNN(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim)
import torchvision
from torchvision.models import resnet18, ResNet18_Weights
model = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

model.fc = torch.nn.Linear(model.fc.in_features, output_dim)


## Training Loop

In [8]:
learning_rate = 0.001
num_epochs = 20
batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    i = 0
    for inputs, labels in train_loader:
        i = i + 1
        print("Training continues..", total_loss, "current i = ", i)    
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Iteration {epoch} - Loss = {total_loss / len(train_loader)}")

Training continues.. 0.0 current i =  1
Training continues.. 1.746855616569519 current i =  2
Training continues.. 3.309792995452881 current i =  3
Training continues.. 4.880220651626587 current i =  4
Training continues.. 6.529273867607117 current i =  5
Training continues.. 8.08720600605011 current i =  6
Training continues.. 9.643678307533264 current i =  7
Training continues.. 11.08583116531372 current i =  8
Training continues.. 12.560041785240173 current i =  9
Training continues.. 14.142629861831665 current i =  10
Training continues.. 15.617859959602356 current i =  11
Training continues.. 17.105870485305786 current i =  12
Training continues.. 18.54550325870514 current i =  13
Training continues.. 19.95233154296875 current i =  14
Training continues.. 21.368703246116638 current i =  15
Training continues.. 22.740788102149963 current i =  16
Training continues.. 24.14760732650757 current i =  17
Training continues.. 25.4925434589386 current i =  18
Training continues.. 26.87389

Training continues.. 186.72222411632538 current i =  149
Training continues.. 187.83550536632538 current i =  150
Training continues.. 189.03896629810333 current i =  151
Training continues.. 190.13075006008148 current i =  152
Training continues.. 191.28031206130981 current i =  153
Training continues.. 192.45986533164978 current i =  154
Training continues.. 193.5710505247116 current i =  155
Training continues.. 194.71099066734314 current i =  156
Training continues.. 195.8633427619934 current i =  157
Training continues.. 197.0173088312149 current i =  158
Training continues.. 198.10335171222687 current i =  159
Training continues.. 199.19085896015167 current i =  160
Training continues.. 200.300173163414 current i =  161
Training continues.. 201.36081850528717 current i =  162
Training continues.. 202.5078890323639 current i =  163
Training continues.. 203.59543538093567 current i =  164
Training continues.. 204.66642117500305 current i =  165
Training continues.. 205.811628818511

Training continues.. 109.15156251192093 current i =  110
Training continues.. 110.11050015687943 current i =  111
Training continues.. 111.05976575613022 current i =  112
Training continues.. 111.98198080062866 current i =  113
Training continues.. 112.91297793388367 current i =  114
Training continues.. 113.82589560747147 current i =  115
Training continues.. 114.74061465263367 current i =  116
Training continues.. 115.72301429510117 current i =  117
Training continues.. 116.61041277647018 current i =  118
Training continues.. 117.52714210748672 current i =  119
Training continues.. 118.44856309890747 current i =  120
Training continues.. 119.39730209112167 current i =  121
Training continues.. 120.30783450603485 current i =  122
Training continues.. 121.19128185510635 current i =  123
Training continues.. 122.12260842323303 current i =  124
Training continues.. 123.01808959245682 current i =  125
Training continues.. 123.99538123607635 current i =  126
Training continues.. 124.903106

Training continues.. 58.40365785360336 current i =  70
Training continues.. 59.22778517007828 current i =  71
Training continues.. 60.08549726009369 current i =  72
Training continues.. 60.86771380901337 current i =  73
Training continues.. 61.68505120277405 current i =  74
Training continues.. 62.459313333034515 current i =  75
Training continues.. 63.29297685623169 current i =  76
Training continues.. 64.07383453845978 current i =  77
Training continues.. 64.86852818727493 current i =  78
Training continues.. 65.71334981918335 current i =  79
Training continues.. 66.50197106599808 current i =  80
Training continues.. 67.29314357042313 current i =  81
Training continues.. 68.174733877182 current i =  82
Training continues.. 69.04663014411926 current i =  83
Training continues.. 69.92380791902542 current i =  84
Training continues.. 70.75690513849258 current i =  85
Training continues.. 71.60781836509705 current i =  86
Training continues.. 72.40612369775772 current i =  87
Training co

Training continues.. 22.747648417949677 current i =  31
Training continues.. 23.538641691207886 current i =  32
Training continues.. 24.336466670036316 current i =  33
Training continues.. 25.084513783454895 current i =  34
Training continues.. 25.86346811056137 current i =  35
Training continues.. 26.67239272594452 current i =  36
Training continues.. 27.46311914920807 current i =  37
Training continues.. 28.22389382123947 current i =  38
Training continues.. 28.98752385377884 current i =  39
Training continues.. 29.727046310901642 current i =  40
Training continues.. 30.471064865589142 current i =  41
Training continues.. 31.205902814865112 current i =  42
Training continues.. 31.97785520553589 current i =  43
Training continues.. 32.728347420692444 current i =  44
Training continues.. 33.45503479242325 current i =  45
Training continues.. 34.324907422065735 current i =  46
Training continues.. 35.03157955408096 current i =  47
Training continues.. 35.781140983104706 current i =  48


...

Training continues.. 32.821988210082054 current i =  114
Training continues.. 33.118157014250755 current i =  115
Training continues.. 33.39539001882076 current i =  116
Training continues.. 33.645716831088066 current i =  117
Training continues.. 33.956654742360115 current i =  118
Training continues.. 34.243557289242744 current i =  119
Training continues.. 34.43738006055355 current i =  120
Training continues.. 34.74340446293354 current i =  121
Training continues.. 35.00891529023647 current i =  122
Training continues.. 35.28398708999157 current i =  123
Training continues.. 35.54011233150959 current i =  124
Training continues.. 35.82975073158741 current i =  125
Training continues.. 36.03945395350456 current i =  126
Training continues.. 36.39219728112221 current i =  127
Training continues.. 36.73138344287872 current i =  128
Training continues.. 37.00275272130966 current i =  129
Training continues.. 37.268420308828354 current i =  130
Training continues.. 37.534405916929245 cu

Training continues.. 19.451183333992958 current i =  75
Training continues.. 19.68850487470627 current i =  76
Training continues.. 19.99281546473503 current i =  77
Training continues.. 20.189094334840775 current i =  78
Training continues.. 20.388849899172783 current i =  79
Training continues.. 20.690735831856728 current i =  80
Training continues.. 20.939394935965538 current i =  81
Training continues.. 21.20924501121044 current i =  82
Training continues.. 21.44964176416397 current i =  83
Training continues.. 21.704194605350494 current i =  84
Training continues.. 21.951638385653496 current i =  85
Training continues.. 22.251316085457802 current i =  86
Training continues.. 22.515822514891624 current i =  87
Training continues.. 22.786127135157585 current i =  88
Training continues.. 23.08752779662609 current i =  89
Training continues.. 23.37385742366314 current i =  90
Training continues.. 23.633190974593163 current i =  91
Training continues.. 23.873645320534706 current i =  9

Training continues.. 8.599604696035385 current i =  36
Training continues.. 8.829400569200516 current i =  37
Training continues.. 9.055344045162201 current i =  38
Training continues.. 9.286520913243294 current i =  39
Training continues.. 9.49713522195816 current i =  40
Training continues.. 9.760874032974243 current i =  41
Training continues.. 10.002883359789848 current i =  42
Training continues.. 10.20496292412281 current i =  43
Training continues.. 10.501023754477501 current i =  44
Training continues.. 10.767222836613655 current i =  45
Training continues.. 11.012203246355057 current i =  46
Training continues.. 11.209746599197388 current i =  47
Training continues.. 11.481045067310333 current i =  48
Training continues.. 11.68498881161213 current i =  49
Training continues.. 11.93308624625206 current i =  50
Training continues.. 12.164920642971992 current i =  51
Training continues.. 12.416637882590294 current i =  52
Training continues.. 12.643344789743423 current i =  53
Tr

Training continues.. 42.87165293097496 current i =  183
Training continues.. 43.125215232372284 current i =  184
Training continues.. 43.309886664152145 current i =  185
Training continues.. 43.60357329249382 current i =  186
Iteration 18 - Loss = 0.23558413782106932
Training continues.. 0.0 current i =  1
Training continues.. 0.18624021112918854 current i =  2
Training continues.. 0.4210888743400574 current i =  3
Training continues.. 0.6317841559648514 current i =  4
Training continues.. 0.8576854020357132 current i =  5
Training continues.. 1.0656472146511078 current i =  6
Training continues.. 1.3115555942058563 current i =  7
Training continues.. 1.5127425640821457 current i =  8
Training continues.. 1.7243865430355072 current i =  9
Training continues.. 1.9363814741373062 current i =  10
Training continues.. 2.1624534130096436 current i =  11
Training continues.. 2.389654576778412 current i =  12
Training continues.. 2.590712606906891 current i =  13
Training continues.. 2.814348

Training continues.. 30.663020074367523 current i =  144
Training continues.. 30.859117060899734 current i =  145
Training continues.. 31.077859669923782 current i =  146
Training continues.. 31.271993950009346 current i =  147
Training continues.. 31.474689856171608 current i =  148
Training continues.. 31.7195942401886 current i =  149
Training continues.. 31.95700316131115 current i =  150
Training continues.. 32.12725281715393 current i =  151
Training continues.. 32.35075867176056 current i =  152
Training continues.. 32.53857287764549 current i =  153
Training continues.. 32.74847835302353 current i =  154
Training continues.. 33.0155935883522 current i =  155
Training continues.. 33.25312311947346 current i =  156
Training continues.. 33.46549180150032 current i =  157
Training continues.. 33.703380808234215 current i =  158
Training continues.. 33.9290771484375 current i =  159
Training continues.. 34.172405794262886 current i =  160
Training continues.. 34.412002965807915 curr

## Evaluation

In [9]:
# Validation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

validation_accuracy = 100 * correct / total
print(f"Validation Accuracy: {validation_accuracy}%")

Validation Accuracy: 88.62075730822052%


In [10]:
# Testing loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy}%")

Test Accuracy: 87.82114140027457%
